# Caution

This notebook relies on Yahoo Finance data.

May give bad values if run during the day, possibly resulting in asynchronous data.

Likely get better values at end-of-day.

In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings

from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13

import sys
sys.path.insert(0, '../cmds')
from options import *

In [2]:
import datetime
import pandas_datareader.data as web
import pandas_datareader as pdr
import yfinance as yf

In [3]:
TICK = 'TSLA'
EXPRY = ['2022-08-19','2022-12-16']

RF_SOFR = False

TODAY = datetime.datetime.today()
TODAY = pd.to_datetime(TODAY)
today_str = TODAY.strftime('%Y-%m-%d')

START_DATE = TODAY - datetime.timedelta(days=5)
END_DATE = TODAY

In [4]:
sec = yf.Ticker(TICK)
px = sec.history('1d',start=START_DATE,end=END_DATE)['Close']

QDATE = px.index[-1]
qdate_label = QDATE.strftime('%Y-%m-%d')

if qdate_label != today_str:
    warnings.warn('Quote date is not as expected.')

In [5]:
### Using SOFR from previous day, as not published until day after
TICKRF = 'SOFR'
sofr = pdr.DataReader(TICKRF, data_source='fred', start=START_DATE,end=END_DATE)

tbill = yf.Ticker('^IRX')
tbill = tbill.history(start=START_DATE, end=END_DATE)[['Close']].rename(columns={'Close':'TBill (3mo)'})
tbill.name = 'TBill (3mo)'

In [6]:
quotes = pd.DataFrame(columns=[sofr.columns[0],tbill.columns[0]], index=[QDATE] + [pd.to_datetime(expry) for expry in EXPRY], dtype=float)

quotes.loc[QDATE] = [sofr.iloc[-1].values/100, tbill.loc[QDATE].values/100]

quotes.loc[QDATE,f'{TICK}'] = px.loc[QDATE]
quotes.index.name = 'date'

# Options

In [7]:
calls= dict()
puts = dict()

for expry in EXPRY:
    opts = sec.option_chain(expry)
    calls[expry], puts[expry] = clean_options(opts.calls, opts.puts)

# Export

In [8]:
outfile = '../data/options_chains_v2.xlsx'
with pd.ExcelWriter(outfile) as writer:  
    quotes.to_excel(writer, sheet_name= f'Market Quotes {qdate_label}', index=True)
    
    for expry in EXPRY:
        calls[expry].to_excel(writer, sheet_name= f'CALLS {expry}', index=True)
        puts[expry].to_excel(writer, sheet_name= f'PUTS {expry}', index=True)